In [23]:
import os
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn import tree
import numpy as np
import pandas as pd
import random
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [24]:
max_T = 500
alpha = 1e-4 # Hyper-  
gamma = 1 # Hyper-
#set random seed to make results reproducible
seed = 1234
np.random.seed(seed)
random.seed(seed)

In [25]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [26]:
experiment_info = {}
experiment_info["K"] = []
experiment_info["budget"] = []
experiment_info["Error"] = []
experiment_info["Equal opportunity difference"] = []
experiment_info["alpha"] = []

# Read Dataset

In [27]:
path = "../predict-responsibly/data/"
dataset = "adult"
protected_feature = "sex"

In [39]:
errors_pre_0 = []
errors_pre_1 = []
errors_post_0 = []
errors_post_1 = []

for K in [1,2,3,4,5]:
    db = np.load(os.path.join(path,dataset,"{}{}_{}.npz".format(dataset,K,protected_feature)))
    x_val = db["x_train"]
    y_val = db["y_train"]
    x_test = db["x_test"]
    y_test = db["y_test"]
    valid_index = np.arange(len(x_val))
    for budget in [1,0.1]:
        for alpha in [1e-4,1e-3,1e-2,1e-1]:
            logits = np.log(db["ydm_train"]/(1-db["ydm_train"]))
            test_logits = np.log(db["ydm_test"]/(1-db["ydm_test"]))
            best_epoch, best_acc = -1,0
            best_logits_valid = logits
            predicted_test_logits = test_logits
            #use budget % of the validation set for training
            valid_index =random.sample(list(valid_index), int(len(valid_index) * budget))


            # get the idxs1 and idxs2
            idxs1 = random.sample(list(valid_index), int(len(valid_index) * 0.7))
            idxs2 = list(set(valid_index) - set(idxs1))
            for t in range(max_T):
                # probs_heldout = sess_run(tf.nn.sigmoid(logits), x_val[idxs2], latent_val[idxs2], sess)
                # logits updates every iteration, thus the probs_heldout is different
                probs_heldout = sigmoid(logits)[idxs2]
                #heldout_loss = np.mean(-y_val[idxs2] * np.log(probs_heldout + 1e-20) - (1-y_val[idxs2]) * np.log(1-probs_heldout + 1e-20))
                heldout_acc =  np.mean((probs_heldout>0.5)==y_val[idxs2])
                # probs = sess_run(tf.nn.sigmoid(logits), x_val, latent_val ,sess)
                # 应为logits是更新的，所以probs也是更新的
                probs = sigmoid(logits)
                val_loss = np.mean(-y_val * np.log(probs + 1e-20) - (1 - y_val) * np.log(1 - probs + 1e-20))
                val_acc = np.mean((probs > 0.5) == y_val)
                # losses.append(val_loss)
                if heldout_acc > best_acc:
                    # print("Update!")
                    best_epoch = t
                    best_acc = heldout_acc
                    best_logits_valid = logits
                    predicted_test_logits = test_logits
                # delta = res(probs,y_val)
                residual = probs - y_val
                for i in range(3):
                    # define the control
                    control_idx1 = db["ydm_train"][idxs1]>0.5
                    control_idx2 = db["ydm_train"][idxs2]>0.5
                    if i==0: 
                        # X_1, f1(x) > 1/2
                        temp_s = control_idx1
                        temp_s_heldout = control_idx2
                    elif i ==1:
                        # X_0, f0(x) ≤ 1/2
                        temp_s = 1- control_idx1
                        temp_s_heldout = 1- control_idx2
                    else:
                        # X
                        temp_s = np.ones_like(control_idx1)
                        temp_s_heldout = np.ones_like(control_idx2)
                    # get the fresh sample for training
                    samples1 = np.where(temp_s == 1)[0]
                    samples2 = np.where(temp_s_heldout == 1)[0]
                    # train the regression model 
                    clf = Ridge(alpha=1) # 如果我不变呢
                    # clf = DecisionTreeRegressor(max_depth = 5)
                    # 如果要把protected feature 去掉 可以考率生成一个新的x_train、valid
                    # 我没有去拟合他的 res 那个function，我直接拟合已知的residual （f(x) - y）
                    clf.fit(x_val[idxs1][samples1],residual[idxs1][samples1])
                    clf_prediction = clf.predict(x_val[idxs2][samples2])
                    corr = np.mean(clf_prediction * residual[idxs2][samples2])
                    # print(t, i, corr)
                    if corr > alpha:
                        # h_models.append(clf)
                        # h = (tf.matmul(latent_ph, tf.constant(np.expand_dims(clf.coef_,-1),
                        #                                         dtype=tf.float32))[:,0] + clf.intercept_)
                        #here, we update h
                        h = clf.predict(x_val)
                        h_test = clf.predict(x_test)
                        # print(h.shape)
                        # print(h)
                        # when update the logits we only update the logits of current set
                        # logits -= .1 * h * s
                        control = db["ydm_train"]>0.5
                        #here ydm_test is the f_0(x_test) 
                        control_test = db["ydm_test"]>0.5
                        if i == 0:
                            # update logits of X_1
                            s = control
                            s_test = control_test
                        elif i == 1:
                            # update logits of X_0
                            s = 1 - control
                            s_test = 1 - control_test
                        else:
                            # update all logits (This might never used)
                            s = np.ones_like(control)
                            s_test = np.ones_like(control_test)
                        logits -= gamma * h * s
                        # update the test logits accordingly
                        test_logits -= gamma * h_test * s_test
                        break
                # 如果 i == 2 说明没有在X_0,X_1找到合适的h，那么就不更新了
                if i==2:
                    break
            #Huangrui add
            EPS = 1e-8
            Y_hat = sigmoid(predicted_test_logits)>0.5
            A = db["attr_test"]
            Y = db["y_test"]
            # print("A",A.shape)
            # print("Y",Y.shape)
            TP = np.multiply(Y, Y_hat)
            mask0 = np.multiply(Y,1-A)
            mask1 = np.multiply(Y,A)
            TP0 = np.multiply(TP, mask0)
            TP1 = np.multiply(TP, mask1)
            tpr0 = np.sum(TP0) / (np.sum(mask0) + EPS)
            tpr1 = np.sum(TP1) / (np.sum(mask1) + EPS)
            #get the error rate of the test set
            error = np.mean(Y_hat != Y)
            eod = np.abs(tpr0 - tpr1)
            print("K={}, budget={}, error={}, eod={}".format(K,budget,error,eod))
            experiment_info["K"].append(K)
            experiment_info["budget"].append(budget)
            experiment_info["Error"].append(error)
            experiment_info["Equal opportunity difference"].append(eod)
            experiment_info["alpha"].append(alpha)
            Ydm = db["ydm_test"]>0.5
            Y_hat0 = Ydm[A==0]
            Y_hat1 = Ydm[A==1]
            Y0 = Y[A==0]
            Y1 = Y[A==1]
            error0 = np.mean(Y_hat0 != Y0)
            error1 = np.mean(Y_hat1 != Y1)
            errors_pre_0.append(error0)
            errors_pre_1.append(error1)
            Y_hat_post0 = Y_hat[A==0]
            Y_hat_post1 = Y_hat[A==1]
            error_post0 = np.mean(Y_hat_post0 != Y0)
            error_post1 = np.mean(Y_hat_post1 != Y1)
            print("error0={}, error1={}".format(error_post0,error_post1))
            errors_post_0.append(error_post0)
            errors_post_1.append(error_post1)``


K=1, budget=1, error=0.17113814364272623, eod=0.19556495506950689
error0=0.20780825302693354, error1=0.09518935516888434
K=2, budget=1, error=0.16383333333333333, eod=0.14347374839032984
error0=0.20438683948155534, error1=0.08199195171026157
K=3, budget=1, error=0.15783333333333333, eod=0.11039119421452698
error0=0.19664328657314628, error1=0.08067729083665338
K=4, budget=1, error=0.16483333333333333, eod=0.12080262081707077
error0=0.20609542842867848, error1=0.08212318477716575
K=5, budget=1, error=0.15769294882480414, eod=0.11281036754094398
error0=0.19621513944223107, error1=0.07967725668179526


In [43]:
#print the pre and pros error for each group
print("For sex = 0, pre error = {}, post error = {}, the difference is {}".format(np.mean(errors_pre_0),np.mean(errors_post_0), np.mean(errors_post_0)-np.mean(errors_pre_0)))
print("For sex = 1, pre error = {}, post error = {}, the difference is {}".format(np.mean(errors_pre_1),np.mean(errors_post_1), np.mean(errors_post_1)-np.mean(errors_pre_1)))

For sex = 0, pre error = 0.20911582724854688, post error = 0.20222978939050895, the difference is -0.00688603785803793
For sex = 1, pre error = 0.08443877843241596, post error = 0.08393180783495205, the difference is -0.0005069705974639072


In [40]:
errors_post_0

[0.20780825302693354,
 0.20438683948155534,
 0.19664328657314628,
 0.20609542842867848,
 0.19621513944223107]

In [31]:
import pandas as pd
df = pd.DataFrame.from_dict(experiment_info)
#save df to csv
df.to_csv("results/adult_sex.csv",index=False)

# let us check based on multiacc metric

In [33]:
# Now get the error and eod of human
eods = []
errors_pre_0 = []
errors_pre_1 = []
errors_post_0 = []
errors_post_1 = []

for K in [1,2,3,4,5]:
    db = np.load(os.path.join(path,dataset,"{}{}_{}.npz".format(dataset,K,protected_feature)))
    Y_hat = db["ydm_test"]>0.5
    A = db["attr_test"]
    Y = db["y_test"]
    Y_hat0 = Y_hat[A==0]
    Y_hat1 = Y_hat[A==1]
    Y0 = Y[A==0]
    Y1 = Y[A==1]
    error0 = np.mean(Y_hat0 != Y0)
    error1 = np.mean(Y_hat1 != Y1)
    errors_pre_0.append(error0)
    errors_pre_1.append(error1)
    